## Agentic AI Framework Project

#### This notebook will showcase the follwoing:

* Agent workflow
* Use of tools to call functions
* Agent collaboration via Tools and Handoffs

In [9]:
import asyncio
from dotenv import load_dotenv
from agents import Agent, trace, Runner, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

### Step1: Agent Workflow

In [3]:
instructions1 = """You are a sales agent working for InsightStat, which is a company offering data science solutions. This \
company leverages the latest AI tools to design tailored data science solutions. Your job is to write cold emails that are serious \
and professional
"""

instructions2 = """You are a humorous, engaging sales agent working for InsightStat, which is a company offering data science solutions. This \
company leverages the latest AI tools to design tailored data science solutions. Your job is to write witty, engaging cold emails will likely get you \
a response"""

instructions3 = """You are a busy ales agent working for InsightStat, which is a company offering data science solutions. This
company leverages the latest AI tools to design tailored data science solutions. Your job is to write serious cold emails that are concise
and to the point"""

In [5]:
sales_agent1 = Agent(
    name = "Professional Sales Agent",
    instructions= instructions1,
    model = 'gpt-4o-mini'
)

sales_agent2 = Agent(
    name = "Engaging Sales Agent",
    instructions= instructions2,
    model = 'gpt-4o-mini'
)

sales_agent3 = Agent(
    name = "Busy Sales Agent",
    instructions= instructions3,
    model = 'gpt-4o-mini'
)

In [7]:
result = Runner.run_streamed(sales_agent1, input= "Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end = "", flush=True)

Subject: Unlock the Power of Data with Tailored Solutions

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am a sales agent with InsightStat, a leader in providing customized data science solutions powered by cutting-edge AI technology.

In today’s competitive landscape, making informed decisions backed by robust data analysis is more crucial than ever. At InsightStat, we specialize in designing tailored solutions that fit the specific needs of your organization, helping you unlock valuable insights and drive better outcomes.

Some of the benefits our clients have experienced include:

- Enhanced predictive analytics for smarter decision-making
- Streamlined processes through automation and optimization
- Deep insights into customer behavior and market trends

I would love the opportunity to discuss how InsightStat can help enhance your data capabilities and support your business objectives. Would you be available for a brief call next week?


In [10]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1,message),
        Runner.run(sales_agent2,message),
        Runner.run(sales_agent3,message)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output, "\n\n")

Subject: Unlock Data-Driven Insights with Tailored Solutions

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am reaching out on behalf of InsightStat, where we specialize in delivering customized data science solutions powered by the latest AI technologies.

In today’s fast-paced business environment, leveraging data effectively can be the key differentiator for success. At InsightStat, we recognize the unique challenges that organizations like yours face and are committed to helping you unlock actionable insights tailored to your specific needs.

Our team of experts collaborates closely with clients across various industries, offering solutions that range from predictive analytics to machine learning models, all designed to enhance decision-making and drive growth.

I would love the opportunity to discuss how InsightStat can support your data initiatives and help you achieve your goals. Would you be available for a brief call next week?

Th

In [11]:
sales_picker = Agent(
    name = "sales_picker",
    instructions = "You have to pick the best cold sales email from the given oprions \
        Imagine you are a customer who will respond to one of the emails that you liked the best \
        Do not give any explanation. Just return the best email",
        model= "gpt-4o-mini" 
)

In [14]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1,message),
        Runner.run(sales_agent2,message),
        Runner.run(sales_agent3,message)
    )

outputs = [result.final_output for result in results]

emails = "Cold sales emails:\n\n" + "\n\nEmail: \n\n".join(outputs)

best = await Runner.run(sales_picker, emails)

print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Unlocking Data Magic – No Hogwarts Required! 🧙‍♂️✨

Hi [Recipient’s Name],

Ever feel like your data is that one magical creature from a fantasy novel—mysterious, elusive, and more likely to cause chaos than help? 🐉 Well, you’re in luck! At InsightStat, we specialize in taming those data dragons and transforming them into valuable insights.

We’ve combined cutting-edge AI with a sprinkle of data science wizardry to create tailored solutions that *actually* make sense of your numbers. Whether you’re trying to predict trends, boost sales, or simply want to understand why your coffee consumption spikes on Monday mornings, we’ve got your back!

Let’s chat about how we can turn your data questions into answers—preferably ones that don’t require crystal balls. You bring the coffee; I’ll bring the magic!

Looking forward to having some fun with your data,

[Your Name]  
[Your Position]  
InsightStat  
[Your Phone Number]  
[Your LinkedIn Profile]  
P.S. We promise n

### Use of Tools

In [15]:
sales_agent1 = Agent(
    name = "Professional Sales Agent",
    instructions= instructions1,
    model = 'gpt-4o-mini'
)

sales_agent2 = Agent(
    name = "Engaging Sales Agent",
    instructions= instructions2,
    model = 'gpt-4o-mini'
)

sales_agent3 = Agent(
    name = "Busy Sales Agent",
    instructions= instructions3,
    model = 'gpt-4o-mini'
)

In [16]:
sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for InsightStat, which is a company offering data science solutions. This company leverages the latest AI tools to design tailored data science solutions. Your job is to write cold emails that are serious and professional\n', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [24]:
from openai import api_key


@function_tool
def send_email(body:str):
    """ Send out an email with the given body to all sales prospects"""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("kausthab.phukan@gmail.com")
    to_email = To("kausthab.phukan@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body = mail)
    return {"status":"success"}

In [18]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001BB9F12E7A0>, strict_json_schema=True, is_enabled=True)

Converting Agent into a tool

In [19]:
tool1 = sales_agent1.as_tool(tool_name= "sales_agent1", tool_description= "Write a cold sales email")

tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001BBA1BA02C0>, strict_json_schema=True, is_enabled=True)

If we can do it for 1, we can do it for all the others

In [20]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name= "sales_agent1", tool_description= description)
tool2 = sales_agent2.as_tool(tool_name= "sales_agent2", tool_description= description)
tool3 = sales_agent3.as_tool(tool_name= "sales_agent3", tool_description= description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001BBA1BA14E0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001BBA1BA0AE0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

### Creating the Sales Manager that will select the best email to send

In [ ]:
instructions = """You are Sales Manager at Insight Stat and responsible for looking over all the communications your sales agents send out to their prospects. \
    It is also your responsibility to select the best cold sales email using the sales agent tool. And this is how you should go about doing it:

    1.  Use all the sales_agent tools to generate three different email drafts. Do not proceed until all the drafts have been prepared and are ready.

    2. Evaluate and draft selection: Evaluate these 3 drafts and select the best/effective amongst these three. You do not need to give any explanation.

    3.  Use the send_email tool to send the selected best email (only the best email) to the prospect 

    Critical rule:

    - You must use the sales_agent tools to write the emails, never write the emails yourself

    - You will use the send_email tool to send only one email to the prospect, and never send more than one email
"""

sales_manager = Agent(name= "sales_manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "send a cold email to a prospect addressing 'Dear CEO' "

with trace("sales manager"):
    result = await Runner.run(sales_manager, message)